In [ ]:
import json
with open("outputs/buckets07-22_16:37.json","r") as file:
  buckets = json.load(file)
print(len(buckets))

In [ ]:

bucket_names_to_bucket={bucket[0]:bucket for bucket in buckets}
display(bucket_names_to_bucket)

In [ ]:
# Prepare batch to come up with names for each bucket
import random
import numpy as np
import json

with open("outputs/buckets_07-28_textsimilarity17.json","r") as file:
  buckets = json.load(file)
bucket_names=list(buckets.keys())
manual_name_to_llm_reponse={}
random.seed(1423)
BATCH_SIZE=1000
bucket_names_batches = np.array_split(bucket_names, range(BATCH_SIZE, len(bucket_names), BATCH_SIZE))
action_i=0
input_len=0
for batch_i in range(len(buckets)//BATCH_SIZE+1):
  with open(f"./outputs/llm_refinement/07-28_textsim17/input/batch{batch_i}.jsonl","w") as file:
    for bucket_name in bucket_names_batches[batch_i]:
      input=\
f"""I am analyzing a dataset full of Congressional actions. I want to refine each action to clean them up and remove the names and descriptions of congresspeople, bills, amendments, times, votes and committees.
Note that there is already some preprocessing to them, I removed all numbers and tried to remove names of committees and representatives where possible, which is why some of them look strange.
Some notes to help you: h r stands for house resolution, p l stands for public or private law, s stands for senate [resolution], amdt stands for amendment. You should remove all trailing spaces.
Here are some example refinements you should base your refinement off of:
Action: debate the house proceeded with minutes of debate on h r 
Refinement: debate the house proceeded with minutes of debate
Action: debate the house proceeded with minutes of debate on h r as amended 
Refinement: debate the house proceeded with minutes of debate
Action: debate the house proceeded with one hour of debate on the walz motion to instruct conferees on h r the instructions contained in the motion seek to direct the managers on the part of the house to resolve all issues and file a conference report not later than june 
Refinement: debate the house proceeded with one hour of debate on the motion
Action: failed of passage/not agreed to in house on agreeing to the resolution failed by the yeas and nays: : present 
Refinement: failed of passage/not agreed to in house on agreeing to the resolution failed by the yeas and nays 
Action: failed of passage/not agreed to in senate: under the order of // having not achieved votes in the affirmative failed of passage in senate by yea nay vote record vote number: 
Refinement: failed of passage/not agreed to in senate having not achieved votes in the affirmative failed of passage in senate by yea nay vote. 
Action: for further action see p l title xv sections through
Refinement: for further action see law
Action: h amdt on agreeing to the johnson e b amendment failed by voice vote
Refinement: amendment on agreeing to the amendment failed by voice vote
Action: h amdt scalise amendment modified by unanimous consent the modification proposes to strike the second instruction of the amendment 
Refinement: amendment modified by unanimous consent
Action: hearings held by the subcommittee on commerce committee and consumer protection prior to introduction 
[A note: this is one subcommittee name, the way I processed the action makes it look like multiple]
Refinement: amendment modified by the subcommittee prior to introduction
Action: point of order that section of the conference report violates the scope of conference under rule xxviii raised in senate 
Refinement: point of order that section of the conference report violates the scope of conference under the rule raised in senate 
Action: read the second time placed on senate legislative calendar under general orders calendar no pursuant to the order of oct 
Refinement: read the second time placed on senate legislative calendar under general orders calendar
Action: rule provides for consideration of h r h r h r and s the resolution provides for consideration of h r h r h r and s under a closed rule with one hour of general debate for each bill the resolution provides for one motion to recommit on h r h r and h r and one motion to commit on s 
Refinement: rule provides for consideration of resolutions. the resolution provides for consideration of resolutions under a closed rule with time for each bill. the resolution provides for one motion to recommit on resolutions and one motion to commit on other resolution
Action: rule provides for one hour of debate on both h r and h r 
Refinement: rule provides for consideration on resolutions
Action: s amdt motion to waive applicable committeeary discipline with respect to amendment sa rejected in senate by yea nay vote record vote number: 
Refinement: amendment motion to waive applicable committeeary discipline with respect to amendment rejected in senate by yea nay vote 
Action: senate insists on its amendment asks for a conference appoints conferees dodd; kerry; lieberman; menendez; shelby; bennett; lugar 
Refinement: senate insists on its amendment asks for a conference appoints conferees
Action: committee civil rights and civil liberties discharged 
Refinement: committee discharged
Action: the committee of the whole proceeded with pro forma amendments 
Refinement: the committee of the whole proceeded with pro forma amendments
Action: the resolution provides for hour of general debate on both bills 
Refinement: the resolution provides for time for general debate on both bills
Action: general leave representative sked unanimous consent that members have legislative days to revise and extend their remarks on h con res agreed to without objection 
Refinement: general leave representative asked unanimous consent that members have legislative days to revise and extend their remarks on the resolution agreed to without objection 
Now, please come up with a refinement for the following action. Please only give me the refinement, nothing else in your response:
{bucket_name}"""
      input_len+=len(input)
      json.dump({"custom_id":"bucket"+str(action_i),"url":"/v1/responses","method":"POST","body":{"input":input,"model":"gpt-4.1-mini"}}, file)
      file.write("\n")
      action_i+=1
    # manual_name_to_llm_reponse[bucket_name]=client.responses.create(input=input,model="gpt-4.1-mini")
    # print(bucket_name)
  # print(input_len)
print(input_len)

In [ ]:
# Load OpenAI client
import dotenv
import os
import openai
import asyncio
import json
import time
dotenv.load_dotenv()
client = openai.Client(api_key=os.environ["OPENAI_API_KEY"])
statuses=[batch.status for batch in client.batches.list()]
while "cancelling" in statuses or "in_progress" in statuses:
  time.sleep(60)
  statuses=[batch.status for batch in client.batches.list()]
print("starting batches")
for batch_i in range(1):
  with open(f"./outputs/llm_refinement/07-28_textsim17/input/batch{batch_i}.jsonl","rb") as file:
    batch_file=client.files.create(file=file,purpose="batch")
  batch=client.batches.create(input_file_id=batch_file.id,endpoint="/v1/responses",completion_window="24h")
  print(batch.id)
  while client.batches.retrieve(batch.id).status!="completed":
    if batch.status=="failed":
      print("failed")
      break
    time.sleep(60)
  with open(f"./outputs/llm_refinement/07-28_textsim17/output/batch{batch_i}.jsonl","w") as file:
    file.write(client.files.content(file_id=client.batches.retrieve(batch_id=batch.id).output_file_id).text)
  print(f"finished batch {batch_i}")

In [ ]:

  with open(f"./outputs/llm_refinement/07-28_textsim17/output/batch{batch_i}.jsonl","w") as file:
    file.write(client.files.content(file_id=client.batches.retrieve(batch_id="batch_6887b2a162b48190a5f8e73a1c470923").output_file_id).text)

In [ ]:
refinements=[]
output_names=open("outputs/bucket_names_07-28_textsim17_refined.txt","w")
output_buckets=open("outputs/buckets_07-28_textsim17_refined.txt","w")
  for batch_i in range(6):
    with open(f"./outputs/llm_refinement/07-28_textsim17/output/batch{batch_i}.jsonl","r") as file:
      for line in file:
        refinement=json.load(line.strip()).content.text
        refinements.append()
        output_file.write(ref)



